In [1]:
import re
import os
import time
import json
import pandas as pd
import requests
import socket
import datetime
import datetime as dt
import warnings
from datetime import datetime
from bs4 import BeautifulSoup as bs
from bs4 import BeautifulSoup
import urllib.request
import newspaper
from newspaper import Article, fulltext
import matplotlib.pyplot as plt
import plotly.plotly as py
import cufflinks as cf
import plotly.tools as tls
from pytrends.request import TrendReq
import pytz
import feedparser


from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
import plotly
import plotly.graph_objs as go

#plotly.offline.init_notebook_mode()
#init_notebook_mode(connected=True)         # initiate notebook for offline plot

#google_username = ""
#google_password = ""
#pytrends = TrendReq()
#cf.go_offline()


#warnings.filterwarnings("ignore")

#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:90% !important; }</style>"))

## Done: Live Coin Watch, Coindesk, Forexlive, Coinlore, Cointelegraph, 

# Live Coin Watch


In [2]:
def getnews(url):

    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"}
    url ='https://news.livecoinwatch.com/'
    res = requests.get(url,headers=headers)

    soup = bs(res.content,'lxml')
    df = pd.DataFrame()
    x=soup.findAll('div',attrs={'class':'td_block_inner td-column-2'})

    Title =[]
    Link =[]
    Text =[]
    Times =[]
    Comment =[]
    for something in x[0].find_all("a"):
        title = something.get('title')
        link = something.get('href')
        Link.append(link)
        Title.append(title)
        foo = something.find_parent('div')
        i = foo.find(class_='td-excerpt')
        if i is not None:
            Text.append(i.text.replace('\r\n',''))
        else:
            Text.append('No Text')   
        b = something.find_parent('div')
        c = b.find('time')
        if c is not None:
            d = c.get('datetime')
            Times.append(pd.to_datetime(d))
        else:
            Times.append('No Time')

        k = something.find_parent('div').find('div',class_='td-module-comments')
        if k is not None:
            Comment.append(k.text)
        else:
            Comment.append('No comment')



    df['Title'] = Title
    df['Text'] = Text
    df['Time'] = Times
    df['Links'] = Link   
    df['Comment'] = Comment

    df = df.dropna().drop_duplicates(subset=['Title'],keep='last')
    return df

In [3]:
url = 'https://news.livecoinwatch.com/'
getnews(url)


,Title,Text,Time,Links,Comment
1,"This Week In Crypto: BTC Tops $10,000, Ripple ...",No Text,2019-06-22 05:11:57+00:00,https://news.livecoinwatch.com/week-crypto-btc...,0
6,New cryptocurrencies on 1xBit: stablecoins com...,One of the leading cryptoc...,2019-06-17 23:00:14+00:00,https://news.livecoinwatch.com/new-cryptocurre...,0
11,This Week In Crypto: Google Lauds Ethereum-bas...,This Week In Crypto is a w...,2019-06-15 18:00:50+00:00,https://news.livecoinwatch.com/this-week-in-cr...,0
16,"This Week In Crypto: India Bitcoin Ban, EOS Vo...",This Week In Crypto is a w...,2019-06-08 04:38:00+00:00,https://news.livecoinwatch.com/this-week-crypt...,0
21,"This Week In Crypto: Bitcoin Bulls Roar, Priva...",This Week In Crypto is a w...,2019-06-01 01:50:08+00:00,https://news.livecoinwatch.com/week-crypto-bit...,0


# Scrape Coindesk

In [4]:
def getnews(url):
    
    res = requests.get(url)
    soup = bs(res.content,'lxml')
    df = pd.DataFrame()
    x=soup.findAll('div',id='body-container')
    b=x[0].find_all('a')
    Title =[]
    for i in b:
        c = i.get('title')
        Title.append(c)
    
    link =[]
    for i in b:
        c = i.get('href')
        link.append(c)
        
    Times =[]
    for i in b:
        time = i.find('time')
        time=str(time)
        time = time.split("\"")[1].split('Z')[0]
        time = pd.to_datetime(time)
        time = time.tz_convert('Europe/Berlin')
        Times.append(time)
        
    Text =[]
    for i in b:
        if i.p is None:
            Text.append('No Text')
        else:
            Text.append(i.p.text)
    
    
    df['Title'] = Title
    df['Text'] = Text
    df['Time']=Times
    df['Links'] = link
    
    return df

In [5]:
url = 'https://www.coindesk.com/'
getnews(url)


,Title,Text,Time,Links
0,A Monumental Fight Over Facebook’s Cryptocurre...,No Text,2019-06-24 06:00:15+02:00,https://www.coindesk.com/a-monumental-fight-ov...
1,Why Ethereum’s Privacy Matters and What’s Bein...,No Text,2019-06-23 11:30:10+02:00,https://www.coindesk.com/why-ethereums-privacy...
2,Staking Isn’t Just a Way to Earn Crypto Money ...,No Text,2019-06-22 11:30:41+02:00,https://www.coindesk.com/staking-isnt-just-a-w...
3,Bitcoin Tops $11.3K to Hit Fresh 2019 Highs,No Text,2019-06-24 03:22:36+02:00,https://www.coindesk.com/bitcoin-tests-11-3k-w...
4,TrendMicro Detects Crypto Mining Malware Affec...,The botnet enters through the Android Debug Br...,2019-06-24 00:30:10+02:00,https://www.coindesk.com/trendmicro-detects-cr...
5,Meeting Bitcoiners Online When They Live in th...,"Bitcoin developer Mark ""Murch"" Erhardt tells C...",2019-06-23 18:15:41+02:00,https://www.coindesk.com/meeting-bitcoiners-on...
6,Howey Schmowey – The Real Answer is to Update ...,The SEC should focus more on fraud prevention ...,2019-06-23 17:30:25+02:00,https://www.coindesk.com/howey-schmowey-the-re...
7,Above $300: Ether Price Clocks 10-Month High,The price of ethereum’s native cryptocurrency ...,2019-06-22 07:32:25+02:00,https://www.coindesk.com/above-300-ether-price...
8,Bitcoin Price Tops $10K for First Time Since 2018,Bitcoin's price passed five digits on cryptocu...,2019-06-22 01:47:58+02:00,https://www.coindesk.com/breaking-bitcoin-pric...
9,‘Don’t Hold Your Breath:’ Australia’s Central ...,Australian regulators don't expect Libra to ga...,2019-06-22 00:00:03+02:00,https://www.coindesk.com/dont-hold-your-breath...


# Scrape Forexlive

In [6]:
def pullnewsforexlive(url):
    df = pd.DataFrame()
    res = requests.get(url)
    soup = bs(res.content,'lxml')
    
    x = soup.find_all('div',class_='col-xs-12 col-sm-7 col-md-8 col-lg-8')
    article =x[0].find_all('article')
    
    Link=[]
    Title =[]
    Text =[]
    Time =[]
    Comment=[]
    for i in article:
        link = i.find('a').get('href')
        title = i.find('a').get('title')
        Link.append(link)
        Title.append(title.replace('View: ','').replace('View Article: ',''))
        time = i.find('time').get('datetime')
        Time.append(pd.to_datetime(time))

        text = i.find(class_='teaser-text')
        if text is not None:
            Text.append(text.text)
        else:
            k = i.find(class_='artbody artbody-limit').get_text(strip=True).replace('\xa0','')
            Text.append(k.replace("\\r","")) 
            
        h= i.find(class_='ccinner')
        Comment.append(int(h.text))
    df['Title'] = Title
    df['Text'] = Text
    df['Time'] = Time
    df['Link'] = Link
    df['Comment'] = Comment
    return df

In [7]:
url = 'https://www.forexlive.com/Cryptocurrency'
pullnewsforexlive(url)

,Title,Text,Time,Link,Comment
0,BoE's Carney to allow crypto companies to depo...,In BoE Governor Carney's maiden House speech h...,2019-06-23 20:26:13+00:00,//www.forexlive.com/cryptocurrency/!/boes-carn...,0
1,Weekend Bitcoin update - look up! Price hits a...,"BTC/USD traded through 10,000 USD over the wee...",2019-06-23 20:18:03+00:00,//www.forexlive.com/cryptocurrency/!/weekend-b...,0
2,"Bitcoin cracks $10K and $11K, and backs off. W...",Watch the depth of the correction. It will tel...,2019-06-22 15:32:19+00:00,//www.forexlive.com/cryptocurrency/!/bitcoin-c...,12
3,Bitcoin eyes $10K. Above the 38.2% of the mov...,"$10,000 in traders view\r\nThe price of bitcoi...",2019-06-21 16:14:08+00:00,//www.forexlive.com/cryptocurrency/!/bitcoin-e...,2
4,Gold and Bitcoin slugging it out for best on f...,I've posted on gold a couple of times - its be...,2019-06-21 02:43:08+00:00,//www.forexlive.com/cryptocurrency/!/gold-and-...,1
5,"If you still have your 'Dow 10,000' hat you mi...",The weakness in the USD is helping to contrib...,2019-06-20 20:49:41+00:00,//www.forexlive.com/cryptocurrency/!/if-you-st...,3
6,QuadrigaCX founder moved customer funds into h...,CEO died in December\r\nCanadian crypto exchan...,2019-06-20 13:28:59+00:00,//www.forexlive.com/cryptocurrency/!/quadrigac...,13
7,"ICYMI - Facebook unveiled its cryptocurrency,...",I posted on Tuesday Asia time the launch was c...,2019-06-19 03:09:47+00:00,//www.forexlive.com/cryptocurrency/!/icymi-fac...,0
8,CFTC lays charges in Bitcoin trading scam that...,UK firm said it earned 1.5% daily\r\nThe CFTC ...,2019-06-18 16:54:35+00:00,//www.forexlive.com/cryptocurrency/!/cftc-lays...,1
9,Get set for Facebook to unveil its cryptocurre...,There are reports from multiple sources that F...,2019-06-18 03:15:14+00:00,//www.forexlive.com/cryptocurrency/!/get-set-f...,2


# Coinlore

In [8]:
def getnews(url):
    res = requests.get(url)
    soup = bs(res.content,'lxml')
    df = pd.DataFrame()
    x=soup.findAll('div',class_='news-item')
    
    General =[]
    for i in x:
        k=i.text
        General.append(k)

    Time =[]
    Title =[]
    Date =[]
    for i in General:
        time = i[1:4].replace('\n','')
        title = i[5:].replace('\n','').replace('\u200a—\u200a','-')
        Time.append(time)
        Title.append(title)
    
    df['Time'] = Time
        
    Date =[]
    for x in df['Time']:
        if 'm' in x:
            x = x.replace('m','')
            time = dt.datetime.now()- dt.timedelta(minutes=int(x))
            Date.append(time)
        elif 'h' in x:
            x = x.replace('h','')
            time = dt.datetime.now()- dt.timedelta(hours=int(x))
            Date.append(time)
    
    df['Time'] = Date
    df['Title'] = Title
    
    return df

In [9]:
url = 'https://www.coinlore.com/crypto-news'
getnews(url)


,Time,Title
0,2019-06-24 09:57:25.155763,ELI9: Azimuth (previously known as NB-PoW) IOTA
1,2019-06-24 09:46:25.155763,‘Insane’ Bitcoin Momentum Goes Overdrive as ‘R...
2,2019-06-24 09:36:25.155763,Announcement Regarding COVA’s Listing on Binan...
3,2019-06-24 09:18:25.155763,TNB reach strategic cooperation with Dididu to...
4,2019-06-24 09:14:25.155763,uPlexa: IoT Mining Contest! 60K UPX In Rewards...
5,2019-06-24 09:13:25.155763,Why iFinex Ethos-Unus Sed Leo ?
6,2019-06-24 09:12:25.155763,"Litecoin (LTC) Spending Avenues Increase, Bull..."
7,2019-06-24 09:12:25.155763,Crypto.com Lists Algorand’s ALGO Crypto.com Ch...
8,2019-06-24 09:12:25.155763,Crypto.com Lists Algorand’s ALGO Crypto.com
9,2019-06-24 09:12:25.155763,Crypto Market Wrap: Tron Flips Stellar to Rega...


# Cointelegraph

In [10]:
def getcointele():
    df = pd.DataFrame()
    d = feedparser.parse('https://cointelegraph.com/rss')

    Link =[]
    Title =[]
    Date=[]
    for i in d.entries:
        Title.append(i['title'])
        Link.append(i['link']) 
        Date.append(pd.to_datetime(i['published']))
    df['Time'] = Date
    df['Title'] = Title
    df['Link'] = Link

    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"}  
    res = requests.get('https://cointelegraph.com/',headers=headers)
    soup = bs(res.content,'lxml')
    x=soup.find_all('p',class_='post-preview-item-card__text')
    Text=[]
    Titlemini = []
    for i in x:
        Text.append(i.text)
        z = i.find_parent('div').find_all('a')
        for i in z:
            if i.get('title') is not None:
                Titlemini.append(i.get('title'))

    df1 = pd.DataFrame()
    df1['Title'] = Titlemini
    df1['Text'] = Text
    df2 = df1.merge(df)

    return df2

In [11]:
getcointele()

,Title,Text,Time,Link
0,Alyx Fashion Brand Launches IOTA-Powered Pilot...,The luxe brand founded by former Lady Gaga cr...,2019-06-24 08:59:00+01:00,https://cointelegraph.com/news/alyx-fashion-br...
1,BIS: Facebook’s Foray into Cryptocurrency Pose...,The Bank of International Settlements warns t...,2019-06-24 08:05:00+01:00,https://cointelegraph.com/news/bis-facebooks-f...
2,Bitcoin Hits 15-Month High as Naysayers Double...,A rollercoaster weekend trading session saw B...,2019-06-24 08:05:00+01:00,https://cointelegraph.com/news/bitcoin-hits-15...
3,A Blockchain System for Azerbaijan’s Digital E...,A Caucasian nation is on the path to digital ...,2019-06-24 01:15:00+01:00,https://cointelegraph.com/news/a-blockchain-sy...
4,"Sharding, Explained","A closer look at the concept of sharding, wha...",2019-06-23 21:28:00+01:00,https://cointelegraph.com/explained/sharding-e...
5,Ethereum Co-Founder Criticizes Facebook’s Libr...,Ethereum co-founder Joseph Lubin said that Fa...,2019-06-23 20:52:00+01:00,https://cointelegraph.com/news/ethereum-co-fou...
6,Craig Wright Failed to Disclose Bitcoin Holdin...,Craig Wright has failed to disclose his bitco...,2019-06-23 19:38:00+01:00,https://cointelegraph.com/news/craig-wright-fa...
7,"Hodler’s Digest, June 17–23: Top Stories, Pric...","The QuadrigaCX story gets weirder, and Satosh...",2019-06-23 19:06:00+01:00,https://cointelegraph.com/news/hodlers-digest-...
8,Report: Two Israeli Brothers Arrested for Hack...,Two Israeli brothers have reportedly been arr...,2019-06-23 18:26:00+01:00,https://cointelegraph.com/news/report-two-isra...
9,Microsoft and Ethereum Foundation Swell the Hy...,"Microsoft, the Ethereum Foundation and Salesf...",2019-06-23 18:06:00+01:00,https://cointelegraph.com/news/microsoft-and-e...


In [15]:
!pip install git+https://github.com/sat28/githubcommit.git


  Cloning https://github.com/sat28/githubcommit.git to c:\users\thuyng~1\appdata\local\temp\pip-req-build-693ad9mp


  Error [WinError 2] The system cannot find the file specified while executing command git clone -q https://github.com/sat28/githubcommit.git C:\Users\THUYNG~1\AppData\Local\Temp\pip-req-build-693ad9mp
Cannot find command 'git' - do you have 'git' installed and in your PATH?
